In [ ]:
host = 'aims2.llnl.gov'
verify = True
api_key = 'TOKEN'

import cwt

client = cwt.WPSClient('https://{!s}/wps/'.format(host), api_key=api_key, verify=verify)
client

In [ ]:
# (122, 240, 480)
inputs = cwt.Variable('http://aims3.llnl.gov/thredds/dodsC/cmip5_css02_data/cmip5/output1/CMCC/CMCC-CM/decadal2005/mon/atmos/Amon/r1i2p1/tas/1/tas_Amon_CMCC-CM_decadal2005_r1i2p1_200511-201512.nc', 'tas')

In [ ]:
proc_subset = client.process_by_name('CDAT.subset')

proc_subset.add_inputs(inputs)

proc_subset.set_domain(cwt.Domain(time=(0, 100), lat=(0, 90)))

proc_subset

In [ ]:
proc_max = client.process_by_name('CDAT.max')

proc_max.add_inputs(proc_subset)

proc_max.add_parameters(axes=['lat'])

proc_max

In [ ]:
proc_workflow = client.process_by_name('CDAT.workflow')
proc_workflow

In [ ]:
client.execute(proc_workflow, [proc_max])

proc_workflow.wait()

In [ ]:
import cdms2

proc_output = [x for x in proc_workflow.output if 'CDAT.max' in x.name][0]

f = cdms2.open(proc_output.uri)

tas = f[proc_output.var_name]

# (3, 480)
print(tas.shape)

In [ ]:
import vcs

v = vcs.init()

v.plot(tas[0])